In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.6 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
texts = [
    "The ebullient sun danced across the azure sky.",
    "Amidst the bustling city, a serene park beckoned.",
    "His laughter echoed through the cobblestone streets.",
    "The ancient tome revealed secrets of forgotten civilizations.",
    "She sipped her tea, lost in the labyrinth of her thoughts.",
    "The moon whispered secrets to the silvery waves.",
    "In the twilight, shadows wove a mysterious tapestry.",
    "The old oak stood as a sentinel of time.",
    "The aroma of freshly baked bread filled the cozy boulangerie.",
    "Beneath the star-studded canvas, dreams took flight.",
    "The whimsical carousel spun its colorful tales.",
    "Lost in the verdant forest, she found solace.",
    "The melancholy notes of a violin echoed.",
    "The effervescent stream flowed over smooth stones.",
    "His eyes held the constellations of distant galaxies.",
    "The enigmatic door led to realms unknown.",
    "The crimson leaves whispered autumn's arrival.",
    "The ethereal mist veiled the ancient ruins.",
    "Her laughter was a symphony in the quiet room.",
    "The iridescent butterfly danced on sunbeams.",
    "The time-worn map revealed hidden treasures.",
    "The whispering wind carried secrets from afar.",
    "The gossamer wings of dragonflies shimmered.",
    "The quixotic quest led to a mythical fountain.",
    "The effulgent sunrise painted the horizon.",
    "Amidst the cacophony of the city, he found peace.",
    "The halcyon days of summer lingered.",
    "The ephemeral beauty of cherry blossoms.",
    "His smile held the warmth of a thousand suns.",
    "The resplendent peacock displayed its plumage.",
    "The luminous moon guided sailors home.",
    "The whirlwind of emotions swept her away.",
    "The sapphire sea stretched to infinity.",
    "The effervescent laughter of children echoed.",
    "The enchanting forest whispered ancient tales.",
    "The serendipitous encounter changed everything.",
    "The incandescent fireflies danced in the night.",
    "The ephemeral snowflakes kissed the ground.",
    "His heart beat in syncopation with hers.",
    "The resolute climber reached the summit."
]

In [ ]:
inputs = tokenizer(texts, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = Dataset(inputs)

In [ ]:
from transformers import TrainingArguments
import accelerate

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=40
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)


In [ ]:
trainer.train()

<ipython-input-9-d87c8321e905>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=120, training_loss=0.005171382427215576, metrics={'train_runtime': 54.4784, 'train_samples_per_second': 29.369, 'train_steps_per_second': 2.203, 'total_flos': 421127700480000.0, 'train_loss': 0.005171382427215576, 'epoch': 40.0})